In [1]:
from LinearTrainer import GridSearcher
from LinearTrainer import LinearTrainer
import torch
import seaborn as sns
sns.set(style="whitegrid")
if torch.cuda.is_available():
    torch.set_float32_matmul_precision('high')


In [2]:
from datasets.utils_datasets import get_mnist_binary_static_loaders
seed = 5
loaders =  get_mnist_binary_static_loaders(seed=seed, batch_size_train=256)

In [3]:
gr = GridSearcher(loaders)

In [4]:
path = 'experiment_1'

In [5]:
gr.conduct_experiment(path=path, do_only_mrvae=False)

Beta-VAE
conducting experiment 1 with beta: 0.010000000000000004 and learning rate: 0.01
using seed: 1


  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
betas = gr.betas

In [ ]:
import pandas as pd

# BETA

In [ ]:
df_beta_vae : pd.DataFrame = pd.read_csv(f'{path}/beta_vae.csv', index_col=0)

In [ ]:
df_beta_vae = df_beta_vae.reset_index(drop=True)
df_beta_vae.head()

In [ ]:
# select the best for beta 
df_beta_vae_sorted = []
for name, group in df_beta_vae.groupby('beta'):
    lr = group.sort_values(by='mean_loss').reset_index().at[0, 'lr']
    df_beta_vae_sorted.append({'beta': name, 'lr': lr})
    print(f'beta={name}, lr={lr}')
df_beta_vae_sorted = pd.DataFrame(df_beta_vae_sorted)

In [ ]:
import torch
import numpy as np
torch.manual_seed(1)
np.random.seed(1)
rate_beta_vae = []
distortion_beta_vae = []
for i in range(0, len(df_beta_vae_sorted)):
    beta = df_beta_vae_sorted.at[i, 'beta']
    lr = df_beta_vae_sorted.at[i, 'lr']
    print(f'best learning rate for beta: {beta} is lr: {lr}\n')
    trainer = LinearTrainer(loaders, use_multi_rate=False, beta=beta, lr=lr)
    trainer.train()
    _, (rate_losses, dist_losses) = trainer.rate_distortion_curve_value(beta, beta)
    rate_beta_vae.append(rate_losses)
    distortion_beta_vae.append(dist_losses)


In [ ]:
import matplotlib.pyplot as plt
# plot the rate distortion curve
fig = plt.figure(figsize=(10, 10))
for i in range(0, len(df_beta_vae_sorted)):
    beta = df_beta_vae_sorted.at[i, 'beta']
    plt.plot(rate_beta_vae[i], distortion_beta_vae[i], 'o', label=f'beta={beta}')
# plt.legend()
plt.xlabel('Rate')
plt.ylabel('Distortion')
plt.title('Rate-Distortion Curve')
try:
    plt.savefig(f'experiment_1/b_vae.pdf', bbox_inches ="tight")
except:
    pass
plt.show()


# MULTI-RATE VAE

In [ ]:
df_multi_rate : pd.DataFrame = pd.read_csv(f'{path}/mr_vae.csv', index_col=0)
df_multi_rate = df_multi_rate.reset_index(drop=True)

In [ ]:
df_multi_rate.head()

In [ ]:
best_lr = df_multi_rate.sort_values(by=['mean_loss']).reset_index().at[0, 'lr']
print(f'best lr for multi-rate vae  is {best_lr}')

In [ ]:
betas

In [ ]:
import torch
import numpy as np
torch.manual_seed(1)
np.random.seed(1)
trainer = LinearTrainer(loaders, use_multi_rate=True, lr=best_lr)
trainer.train()


In [ ]:
betas = np.linspace(start=np.log(0.01), stop=np.log(10.), num=30)

In [ ]:
rate_mr_vae = []
distortion_mr_vae = []
for b in np.linspace(start=np.log(0.01), stop=np.log(10.), num=30):
    _, (rate_losses, dist_losses) = trainer.rate_distortion_curve_value(beta_in=b, beta_loss=1.)
    rate_mr_vae.append(rate_losses)
    distortion_mr_vae.append(dist_losses)

In [ ]:
import matplotlib.pyplot as plt
# plot the rate distortion curve
fig = plt.figure(figsize=(10, 10))
for i in range(0, len(betas)):
    beta = betas[i]
    plt.plot(rate_mr_vae[i], distortion_mr_vae[i], 'o', label=f'beta={beta}')
# plt.legend()
plt.xlabel('Rate')
plt.ylabel('Distortion')
plt.title('Rate-Distortion Curve')
try:
    plt.savefig(f'experiment_1/mr_vae.pdf', bbox_inches ="tight")
except:
    print('warning saving pdf image')
    pass
plt.show()